In [2]:
# Cell 1 - imports & config
import os
import json
import time
import re
import random
from pathlib import Path
from typing import List, Tuple, Dict
from tqdm.auto import tqdm

import requests
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# local config
SAMUDRIK_TXT = "hasta_samudrik_cleaned.txt"   # path to your book text
OUTPUT_JSONL = "json_to_interpretation_pairs.jsonl"
INDEX_DIR = "faiss_index"
CHUNK_SIZE = 800            # characters per chunk
CHUNK_OVERLAP = 200
EMB_MODEL = "all-MiniLM-L6-v2"   # small & fast
EMB_DIM = 384
TOP_K = 5
NUM_PAIRS = 1000            # target number of pairs to produce
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "mistral:7b-instruct-q4_0")  # set to the model you pulled
BATCH_JSONS_PER_CALL = 10   # how many random JSONs to request per Ollama call
OLLAMA_SLEEP = 0.08         # small delay between Ollama requests
RAG_TEMPERATURE = 0.2       # low temp for grounded interpretations
RNG_SEED = 42

# numeric ranges that match typical CV outputs; tune as needed
BREAKS_RANGE = (0, 10)
BRANCHES_RANGE = (0, 300)

random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

Path(INDEX_DIR).mkdir(exist_ok=True)


c:\Users\SID\OneDrive\Desktop\AIML Projs\Palmistry CV\palmistry_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import re
from typing import List

CHUNK_SIZE = 800  # adjust if needed
CHUNK_OVERLAP = 100

def chunk_file(path: str, chunk_size: int = CHUNK_SIZE, overlap: int = CHUNK_OVERLAP) -> List[str]:
    chunks = []
    buffer = ""

    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            buffer += line.strip() + " "
            while len(buffer) > chunk_size:
                # cut at chunk_size
                j = chunk_size
                # try to cut nicely at sentence boundary
                tail = buffer[j:j+200]
                m = re.search(r'([.?!]\s)', tail)
                if m:
                    j = j + m.end()
                chunk = buffer[:j].strip()
                chunks.append(chunk)
                # prepare buffer for next chunk (with overlap)
                buffer = buffer[j - overlap:]

    # add any leftover
    if buffer.strip():
        chunks.append(buffer.strip())

    return chunks

chunks = chunk_file(SAMUDRIK_TXT)
print(f"Created {len(chunks)} chunks.")


Created 750 chunks.


In [4]:
# Cell 3 - embeddings + FAISS index
embedder = SentenceTransformer(EMB_MODEL)

INDEX_PATH = Path(INDEX_DIR) / "index.faiss"
META_PATH = Path(INDEX_DIR) / "meta.json"

def build_or_load_index(chunks: List[str]):
    if INDEX_PATH.exists() and META_PATH.exists():
        print("Loading existing FAISS index & metadata ...")
        index = faiss.read_index(str(INDEX_PATH))
        with open(META_PATH, "r", encoding="utf-8") as f:
            metas = json.load(f)
        return index, metas
    print("Computing embeddings and building FAISS index (this may take a moment)...")
    embeddings = []
    batch = 64
    for i in range(0, len(chunks), batch):
        batch_texts = chunks[i:i+batch]
        emb = embedder.encode(batch_texts, show_progress_bar=False, convert_to_numpy=True)
        embeddings.append(emb)
    embeddings = np.vstack(embeddings).astype('float32')
    # normalize for IP search
    faiss.normalize_L2(embeddings)
    index = faiss.IndexFlatIP(EMB_DIM)
    index.add(embeddings)
    faiss.write_index(index, str(INDEX_PATH))
    metas = [{"id": idx, "text": chunks[idx]} for idx in range(len(chunks))]
    with open(META_PATH, "w", encoding="utf-8") as f:
        json.dump(metas, f, ensure_ascii=False, indent=2)
    print("Index built and saved.")
    return index, metas

index, metas = build_or_load_index(chunks)


Loading existing FAISS index & metadata ...


In [5]:
# Cell 4 - retrieval helper
def retrieve_passages_for_json(json_input: Dict, top_k: int = TOP_K) -> List[Tuple[int, float, str]]:
    # build simple text query summarizing JSON features
    parts = []
    for ln, info in json_input["lines"].items():
        if not info.get("present"):
            parts.append(f"{ln}:absent")
        else:
            parts.append(f"{ln}:{info.get('length','')},{info.get('clarity','')},breaks={info.get('breaks',0)},branches={info.get('branches',0)}")
    query = " ; ".join(parts)
    q_emb = embedder.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)
    idxs, scores = I[0], D[0]
    results = []
    for idx, score in zip(idxs, scores):
        if idx < 0:
            continue
        results.append((int(idx), float(score), metas[idx]["text"]))
    return results

# quick test retrieval
test_json = {"lines": {"life_line": {"present": True, "clarity": "clear", "length": "long", "breaks": 1, "branches": 5},
                       "head_line": {"present": False, "clarity": "", "length": "", "breaks": 0, "branches": 0},
                       "heart_line": {"present": False, "clarity": "", "length": "", "breaks": 0, "branches": 0},
                       "fate_line": {"present": False, "clarity": "", "length": "", "breaks": 0, "branches": 0}}}
print("Sample retrieval (top2):")
for pid, sc, txt in retrieve_passages_for_json(test_json, top_k=2):
    print(pid, sc, txt[:200].replace("\n"," ") + "...")


Sample retrieval (top2):
208 0.594800591468811 , in many cases, death, unless this break is supported bv a sister line or is enclosed in a square. But the break should be verified from both the hands. If the line is laddered and broken up, a perio...
619 0.55412757396698 him with the gifts of organization and administration denoted, by a straight line of Head. Fic. 60. THE HAND OF A POLITICIAN AND ADMINISTRATOR 1. Double line of Life. 2. More than three up-going lines...


In [6]:
# Cell 5 - Ollama call and parse utilities
OLLAMA_GENERATE_ENDPOINT = OLLAMA_URL.rstrip("/") + "/api/generate"
JSON_ARRAY_RE = re.compile(r'\[.*\]', re.S)
JSON_OBJ_RE = re.compile(r'\{.*\}', re.S)

def call_ollama(prompt: str, model: str = OLLAMA_MODEL, max_tokens: int = 1024, temperature: float = 1.0, timeout: int = 60) -> str:
    payload = {
        "model": model,
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stream": False   # 👈 IMPORTANT
    }
    r = requests.post(OLLAMA_GENERATE_ENDPOINT, json=payload, timeout=timeout)
    r.raise_for_status()
    dd = r.json()
    
    if isinstance(dd, dict) and "response" in dd:
        return dd["response"]
    return str(dd)


def extract_json_list_from_text(text: str) -> List[dict]:
    """Try to extract a JSON array from text; if only object found, wrap it into a list."""
    text = text.strip()
    m = JSON_ARRAY_RE.search(text)
    if m:
        s = m.group(0)
    else:
        m2 = JSON_OBJ_RE.search(text)
        s = m2.group(0) if m2 else text
        if s and not s.strip().startswith("["):
            s = "[" + s + "]"
    try:
        parsed = json.loads(s)
        if isinstance(parsed, dict):
            return [parsed]
        return parsed
    except Exception as e:
        # attempt simple fixes: single quotes -> double quotes
        try:
            fixed = s.replace("'", '"')
            parsed = json.loads(fixed)
            return parsed if isinstance(parsed, list) else [parsed]
        except Exception:
            raise RuntimeError(f"Failed to parse JSON from Ollama output. Raw text:\n{s[:1000]}\nError: {e}")


In [7]:
# Cell 6 - prompt builders

def prompt_generate_jsons(batch_n: int = BATCH_JSONS_PER_CALL) -> str:
    # ask for an array of JSON objects to speed up generation
    return f"""
Generate exactly {batch_n} RANDOM JSON objects, and return them as a single JSON ARRAY (i.e. start with '[' and end with ']').
Each object must follow this EXACT schema (use only these fields and types):

{{ "lines": {{
  "life_line":  {{ "present": true|false, "clarity": "clear|faint|broken", "length": "short|medium|long", "breaks": <int>, "branches": <int> }},
  "head_line":  {{ "present": true|false, "clarity": "clear|faint|broken", "length": "short|medium|long", "breaks": <int>, "branches": <int> }},
  "heart_line": {{ "present": true|false, "clarity": "clear|faint|broken", "length": "short|medium|long", "breaks": <int>, "branches": <int> }},
  "fate_line":  {{ "present": true|false, "clarity": "clear|faint|broken", "length": "short|medium|long", "breaks": <int>, "branches": <int> }}
}} }}

Rules:
- If 'present' is false for a line, then 'clarity' and 'length' MUST be empty strings (""), and 'breaks' & 'branches' MUST be 0.
- If 'present' is true, pick clarity from (clear,faint,broken), length from (short,medium,long).
- Use realistic integers: breaks (0-12), branches (0-500).
- Return ONLY the JSON ARRAY and nothing else (no commentary, no numbering).
- Produce varied combinations (not the same repeated object).
"""
def prompt_rag_interpretation(json_input: dict, passages: List[Tuple[int,float,str]]) -> str:
    # Build prompt that includes retrieved passages and strict instruction to use only them
    header = ("You are an expert in traditional Indian palmistry (Hasta Samudrik Shastra). "
              "Using ONLY the passages provided below (do not invent or add extra classical references), "
              "write a concise interpretation (2-6 sentences) for the following palm features in JSON form. "
              "If no passage directly supports a certain feature, do not assert it.\n\n")
    json_block = "Palm features (JSON):\n" + json.dumps(json_input, ensure_ascii=False, indent=2) + "\n\n"
    passages_block = "Retrieved passages (id => excerpt):\n"
    for pid, score, text in passages:
        excerpt = text.replace("\n", " ").strip()
        if len(excerpt) > 800:
            excerpt = excerpt[:800] + "..."
        passages_block += f"PASSAGE {pid} (score={score:.4f}): {excerpt}\n\n"
    instruction = ("\nNow provide a concise, grounded interpretation in 2-6 sentences. Use only the passages above. "
                   "Start the interpretation with 'Interpretation:' and then the content. Return just the interpretation text.")
    return header + json_block + passages_block + instruction


In [8]:
# Cell 7 - normalization and sanity check
LENGTH_CHOICES = {"short","medium","long"}
CLARITY_CHOICES = {"clear","faint","broken"}

def normalize_json_obj(j: dict) -> dict:
    """Ensure object matches schema strictly; coerce types and clamp numeric ranges."""
    out = {"lines": {}}
    for ln in ["life_line","head_line","heart_line","fate_line"]:
        v = (j.get("lines") or {}).get(ln)
        if not v or not bool(v.get("present")):
            out["lines"][ln] = {"present": False, "clarity": "", "length": "", "breaks": 0, "branches": 0}
            continue
        # present True
        clarity = str(v.get("clarity","")).strip().lower()
        length = str(v.get("length","")).strip().lower()
        if clarity not in CLARITY_CHOICES:
            clarity = random.choice(list(CLARITY_CHOICES))
        if length not in LENGTH_CHOICES:
            length = random.choice(list(LENGTH_CHOICES))
        try:
            breaks = int(v.get("breaks", 0))
        except:
            breaks = random.randint(*BREAKS_RANGE)
        try:
            branches = int(v.get("branches", 0))
        except:
            branches = random.randint(*BRANCHES_RANGE)
        # clamp numeric ranges
        breaks = max(BREAKS_RANGE[0], min(BREAKS_RANGE[1], breaks))
        branches = max(BRANCHES_RANGE[0], min(BRANCHES_RANGE[1], branches))
        out["lines"][ln] = {"present": True, "clarity": clarity, "length": length, "breaks": breaks, "branches": branches}
    return out

def validate_json_schema(j: dict) -> bool:
    try:
        assert isinstance(j, dict) and "lines" in j
        for ln in ["life_line","head_line","heart_line","fate_line"]:
            v = j["lines"][ln]
            assert isinstance(v["present"], bool)
            assert isinstance(v["clarity"], str)
            assert isinstance(v["length"], str)
            assert isinstance(v["breaks"], int)
            assert isinstance(v["branches"], int)
        return True
    except Exception:
        return False


In [ ]:
# Cell 8 - main generation loop
def produce_jsonl_pairs(target_n: int = NUM_PAIRS, out_file: str = OUTPUT_JSONL,
                        batch_size: int = BATCH_JSONS_PER_CALL, top_k: int = TOP_K):
    produced = 0
    seen_inputs = set()
    Path(out_file).parent.mkdir(parents=True, exist_ok=True)
    with open(out_file, "w", encoding="utf-8") as fout:
        pbar = tqdm(total=target_n, desc="Generating pairs")
        while produced < target_n:
            # 1) request a batch of random JSONs from Ollama
            try:
                prompt = prompt_generate_jsons(batch_n=batch_size)
                raw = call_ollama(prompt, model=OLLAMA_MODEL, max_tokens=2048, temperature=1.0)
                objs = extract_json_list_from_text(raw)
            except Exception as e:
                print("Ollama JSON generation error:", e)
                # fallback to simple python sampling if Ollama fails
                objs = []
                for _ in range(batch_size):
                    objs.append({
                        "lines":{
                            ln: ({"present": False, "clarity": "", "length": "", "breaks":0, "branches":0} 
                                 if random.random() < 0.15 else
                                 {"present": True,
                                  "clarity": random.choice(list(CLARITY_CHOICES)),
                                  "length": random.choice(list(LENGTH_CHOICES)),
                                  "breaks": random.randint(*BREAKS_RANGE),
                                  "branches": random.randint(*BRANCHES_RANGE)})
                            for ln in ["life_line","head_line","heart_line","fate_line"]
                        }
                    })
            # 2) normalize & dedup
            normalized_batch = []
            for obj in objs:
                try:
                    norm = normalize_json_obj(obj)
                    key = json.dumps(norm, sort_keys=True)
                    if key in seen_inputs:
                        continue
                    seen_inputs.add(key)
                    normalized_batch.append(norm)
                except Exception as e:
                    continue

            # 3) for each normalized json do RAG retrieve and ask Ollama to interpret
            for j_in in normalized_batch:
                if produced >= target_n:
                    break
                # retrieve passages
                passages = retrieve_passages_for_json(j_in, top_k=top_k)
                # build prompt and generate interpretation (low temperature)
                rag_prompt = prompt_rag_interpretation(j_in, passages)
                try:
                    interp_raw = call_ollama(rag_prompt, model=OLLAMA_MODEL, max_tokens=1024, temperature=RAG_TEMPERATURE)
                except Exception as e:
                    print("Ollama error on interpretation:", e)
                    interp_raw = ""
                # best effort: try to extract the interpretation text (we asked to start with 'Interpretation:')
                interp = interp_raw.strip()
                # If the model returned JSON or extra, try to isolate 'Interpretation:' line
                m = re.search(r'Interpretation:\s*(.+)', interp, re.S)
                if m:
                    interp = m.group(1).strip()
                # fallback: if empty, set a placeholder
                if not interp:
                    interp = "(No interpretation generated - fallback) "

                # save entry
                entry = {
                    "input": j_in,
                    "output": interp,
                    "retrieved": [{"id": pid, "score": score, "excerpt": txt[:800]} for pid, score, txt in passages]
                }
                fout.write(json.dumps(entry, ensure_ascii=False) + "\n")
                produced += 1
                pbar.update(1)
                time.sleep(OLLAMA_SLEEP)  # small delay to be polite
        pbar.close()
    print(f"Finished. Wrote {produced} pairs to {out_file}")

# run
produce_jsonl_pairs(target_n=NUM_PAIRS, out_file=OUTPUT_JSONL, batch_size=BATCH_JSONS_PER_CALL, top_k=TOP_K)


Generating pairs:   0%|          | 0/1000 [00:00<?, ?it/s]

Ollama JSON generation error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=60)


Generating pairs:   0%|          | 1/1000 [02:04<34:27:26, 124.17s/it]

Ollama error on interpretation: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=60)


Generating pairs:   0%|          | 3/1000 [03:57<20:02:50, 72.39s/it] 

...

Ollama error on interpretation: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=60)


Generating pairs:  12%|█▏        | 119/1000 [2:04:05<12:59:23, 53.08s/it]

Ollama error on interpretation: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=60)


Generating pairs:  12%|█▏        | 120/1000 [2:05:08<13:38:30, 55.81s/it]

Ollama error on interpretation: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=60)


In [1]:
# Cell 9 - inspect a few random examples
def sample_jsonl(path: str, k: int = 5):
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print(f"Total entries: {len(lines)}")
    import random
    for item in random.sample(lines, min(k, len(lines))):
        j = json.loads(item)
        print("INPUT:")
        print(json.dumps(j["input"], indent=2, ensure_ascii=False))
        print("RETRIEVED (top2):")
        for r in j.get("retrieved", [])[:2]:
            print("  id:", r["id"], "score:", r["score"])
            print("  excerpt:", r["excerpt"][:300].replace("\n"," "), "...")
        print("OUTPUT:")
        print(j["output"])
        print("-"*60)

# sample a few
sample_jsonl(OUTPUT_JSONL, k=3)


NameError: name 'OUTPUT_JSONL' is not defined